In [ ]:
import hashlib # hash 함수용 sha256 사용할 라이브러리
import json
import time
import random
import requests
from flask import Flask, request, jsonify

7개의 함수: _init_, hash, last_block, pos, pick winner, new_block, new_transaction
- 노드 객체 선언 전 PoS 진행을 위해 해당 노드의 예치 비중 값을 함께 저장
- 채굴 부분이 변경됨 
이 두 가지가 차이점

In [ ]:
class Blockchain(object):
    
    def __init__(self, account_name, account_weight): # 첫 블록체인 객체 생성시 동작
        self.chain = []                                   # chain에 여러 block들 들어옴
        self.current_transaction = []                     # 임시 transaction 넣어줌
        self.nodes = set()                                # Node 목록을 보관
        self.miner_wallet = {'account_name': account_name, 'weight': account_weight}  # 지갑정보 생성
        self.new_block(previous_hash='genesis_block', address = account_name)        # genesis block 생성
        self.account_name = account_name
        self.account_weight = account_weight

        
    @staticmethod # sha256 방식에 의한 해시암호화 진행 
    def hash(block):
        block_string = json.dumps(block, sort_keys=True).encode() 
        return hashlib.sha256(block_string).hexdigest()   # hash 라이브러리로 sha256 사용
    
    @property # 가장 마지막 블록의 정보 확인 
    def last_block(self):
        return self.chain[-1]                             # 체인의 마지막 블록 가져오기!!


    def pos(self): # pick_winner 함수를 활용해 검증자 선정 
        winner_list = []            # 각 노드에서 pick_winner 결과 뽑힌 winner 리스트
        time.sleep(1)
        my_winner = self.pick_winner(account_name = self.account_name, account_weight = self.account_weight)   
        winner_list.append(my_winner)   # winner 리스트에 내노드 결과 넣기
        time.sleep(1)
        
        for target_node in blockchain.nodes:            # 다른 노드들도 pick_winner 진행 
            print(target_node)
            headers = {'Content-Type' : 'application/json; charset=utf-8'}
            res = requests.get('http://' + target_node   + "/nodes/pick_winner", headers=headers)
            winner_info = json.loads(res.content)  # 근처 노드들 선정결과 받아와서
            print(winner_info)
            winner_list.append(winner_info['winner']) 

        final_winner = max(winner_list,key = winner_list.count)  # 각 노드들의 pos 결과로 가장 많이 선정된 winner를 최종 winner 로 선정
        print("final_winner selected : ", final_winner)
        
        return final_winner
            
        
    def pick_winner(self,account_name, account_weight):  ### 누가누가 블록 만들래!! 만들사람 뽑기
        # pos에 사용되는 함수. 모든 노드의 비중을 확률로 해서 검증자를 선정해서 준다.
        candidate_list = []  # POS 대상자를 뽑을 전체 풀!!
             
        for w in range(account_weight):  # 나의 노드들의 weight 수만큼 추가
            candidate_list.append(account_name)
       
        random.shuffle(candidate_list)       #  랜덤으로 섞고!
        for x in  candidate_list:           #  첫번째 node를 winner로 선정
            winner  = x
            print("WINNER SELECTED : ", winner)
            break
        
        return winner                       # winner 공개
    
    
        
    def new_transaction(self, sender, recipient, amount, smart_contract):
        # 거래 내역 리스트에 새로운 거래내역 추가. 블록 생성시 해당 거래 내역 리스트는 블록에 저장된 후 초기화됨
        self.current_transaction.append(
            {
                'sender' : sender, # 송신자
                'recipient' : recipient, # 수신자
                'amount' : amount, # 금액
                'timestamp':time.time(),
                'smart_contract' : smart_contract
            }
        )
        return self.last_block['index'] + 1   

    def new_block(self, previous_hash=None, address = ''):
        # 새로운 거래 내역 리스트를 블록에 저장. PoW에서의 논스가 사라지고 검증자 정보 추가 
        block = {
            'index' : len(self.chain)+1,
            'timestamp' : time.time(), # timestamp from 1970
            'transactions' : self.current_transaction,
            'previous_hash' : previous_hash ,
            'validator' : address
        }
        block["hash"] = self.hash(block)
        self.current_transaction = []
        self.chain.append(block)     
        return block

Blockchain 객채를 기반으로 노드 생성
- 아래는 1개 노드로 먼저 운영하여, 100개의 pyETH를 예치하여 시작한 것으로 가정

In [ ]:
my_ip = '0.0.0.0'
my_port = '5000'
node_identifier = 'node_'+my_port
mine_owner = 'master'
mine_profit = 0.1

blockchain = Blockchain(account_name=mine_owner, account_weight= 100)

app = Flask(__name__)

@app.route('/chain', methods=['GET'])
def full_chain():
    print("chain info requested!!")
    response = {
        'chain' : blockchain.chain, 
        'length' : len(blockchain.chain), 
    }
    return jsonify(response), 200

@app.route('/transactions/new', methods=['POST'])
def new_transaction():
    values = request.get_json() 
    print("transactions_new!!! : ", values)
    required = ['sender', 'recipient', 'amount'] 

    if not all(k in values for k in required):
        return 'missing values', 400
    
    if 'smart_contract' not in values:
        values['smart_contract'] = 'empty'

    index = blockchain.new_transaction(values['sender'],values['recipient'],
values['amount'], values['smart_contract'])
        
    response = {'message' : 'Transaction will be added to Block {%s}' % index}
    return jsonify(response), 201


@app.route('/mine', methods=['GET'])
def mine():
    print("MINING STARTED")    
    final_winner = blockchain.pos()  
    
    if final_winner == blockchain.account_name:  # 만약 본 노드가 winner로 선정되었으면 아래와 같이

        blockchain.new_transaction(            #  나에게 보상을 주고
            sender=mine_owner, 
            recipient=node_identifier, 
            amount=mine_profit, # coinbase transaction 
            smart_contract={"contract_address":"mining_profit"}, 
        )

        previous_hash = blockchain.hash(blockchain.chain[-1])
        block = blockchain.new_block(previous_hash = previous_hash, address = mine_owner)  #  신규 블록 생성
        print("MY NODE IS SELECTED AS MINER NODE")

        response = {
            'message' : 'new block found',
            'index' : block['index'],
            'transactions' : block['transactions'],
            'nonce' : block['validator'],
            'previous_hash' : block['previous_hash'],
            'hash' : block['hash']
        }

        return jsonify(response), 200
    
    else : # isWinner = False : 본 노드가 winner가 아님
        print("MY NODE IS NOT SELECTED AS MINER NODE")

        response = {
            'message' : 'NOT SELECTED'
        }

        return jsonify(response), 200
    
if __name__ == '__main__':
    app.run(host=my_ip, port=my_port)